In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

This notebook shows some basics stats about securities labeled as NC1

## Load Data

In [2]:
data = pd.DataFrame([])
for n in range(2,9):
    file_name = 'trimmed_'+ str(2000+n) +'_v2.csv'
    data = data.append(pd.read_csv(file_name),ignore_index=True)

In [3]:
#Get all rows with NC1
nc1 = data.loc[data['Label'] == 'NC1(failing)']

## Data Validation

In [4]:
print('we have {} rows of data'.format(data.shape[0]))
print('we have {} rows of NC1'.format(nc1.shape[0]))
print('percentage: {:.2f}%'.format(nc1.shape[0]/data.shape[0]*100))

we have 24597 rows of data
we have 3856 rows of NC1
percentage: 15.68%


In [5]:
valid = nc1.groupby(by=['Year','Label'])['CUSIP','PID'].size().sum() == nc1.shape[0]
print("Does nc1.groupby(by=['Year','Label'])['CUSIP','PID'] equal number of rows in nc1? {}".format(valid))

Does nc1.groupby(by=['Year','Label'])['CUSIP','PID'] equal number of rows in nc1? True


In [6]:
valid = data.groupby(by=['Year','Label'])['CUSIP','PID'].size().sum() == data.shape[0]
print("Does data.groupby(by=['Year','Label'])['CUSIP','PID'] equal number of rows in data? {}".format(valid))
if (valid): print("This ensures ['CUSIP','PID'] shows uniqueness.")

Does data.groupby(by=['Year','Label'])['CUSIP','PID'] equal number of rows in data? True
This ensures ['CUSIP','PID'] shows uniqueness.


In [7]:
print('Number of NC1 in given year. Note:2008 does not have NC1')
nc1.groupby(by=['Year','Label'])['CUSIP','PID'].size()

Number of NC1 in given year. Note:2008 does not have NC1


Year  Label       
2002  NC1(failing)       5
2004  NC1(failing)     461
2005  NC1(failing)    1402
2006  NC1(failing)       8
2007  NC1(failing)    1980
dtype: int64

In [8]:
print('Ratio of NC1/All in given year:')
nc1.groupby(by=['Year','Label'])['CUSIP','PID'].size()/data.groupby(by=['Year'])['CUSIP','PID'].size()

Ratio of NC1/All in given year:


Year  Label       
2002  NC1(failing)    0.001805
2004  NC1(failing)    0.103596
2005  NC1(failing)    0.279561
2006  NC1(failing)    0.001990
2007  NC1(failing)    0.444345
dtype: float64

In [9]:
valid = (data.loc[data['Label'] != 'NC1(failing)'].shape[0]) == data.shape[0]-nc1.shape[0]
print("Is data.loc[data['Label'] != 'NC1(failing)'] valid for geting non-NC1? {}".format(valid))

Is data.loc[data['Label'] != 'NC1(failing)'] valid for geting non-NC1? True


## Explore NC1

In [10]:
count = nc1.groupby('PID')['CUSIP'].nunique().count()
print('There are ' + str(count) + ' of prospectus that have NC1 with unique CUSIP')
print("On average {:.2f} NC1 per prospectus that has NC1".format(nc1.shape[0]/count))

There are 669 of prospectus that have NC1 with unique CUSIP
On average 5.76 NC1 per prospectus that has NC1


In [11]:
#These two are different because CUSIP is not unique
#nc1.groupby('PID')['CUSIP'].size().sum()
#nc1.groupby('PID')['CUSIP'].nunique()

In [12]:
show_top = 5
print('Top {} of the MTG_TRANCHE_TYP_LONG among NC1 are:'.format(show_top))
nc1.groupby(['MTG_TRANCHE_TYP_LONG'])['CUSIP','PID'].size().sort_values(ascending=False).head(show_top)

Top 5 of the MTG_TRANCHE_TYP_LONG among NC1 are:


MTG_TRANCHE_TYP_LONG
SUB,CSTR,NAS            1115
SUB,NAS                  469
MEZ,FLT,STEP,IRC         459
MEZ,FLT,STEP             166
MEZ,FLT,STEP,IRC,NAS     163
dtype: int64

In [13]:
print('Top {} of the MTG_TRANCHE_TYP_LONG among non-NC1s are:'.format(show_top))
data.loc[data['Label'] != 'NC1(failing)'].groupby(['MTG_TRANCHE_TYP_LONG'])['CUSIP','PID'].size().sort_values(ascending=False).head(show_top)

Top 5 of the MTG_TRANCHE_TYP_LONG among non-NC1s are:


MTG_TRANCHE_TYP_LONG
SUB,CSTR,NAS        1720
SEQ,AS              1468
MEZ,FLT,STEP,IRC    1466
SUB,NAS             1335
FLT,STEP,IRC         920
dtype: int64

[See here for more info about MTG_TRANCHE_TYP](https://docs.google.com/spreadsheets/d/1MOwPnTr2owqPoJNy73U7UEc3z1RvtzELOCM0ZFxBJU8/edit?usp=sharing)

In [14]:
nc1_total = nc1['MTG ORIG AMT'].sum()
nonNC1_total = data.loc[data['Label'] != 'NC1(failing)']['MTG ORIG AMT'].sum()
print('Sum of MTG ORIG AMT among NC1 = {:.2f}'.format(nc1_total))
print('Sum of MTG ORIG AMT among non-NC1 = {:.2f}'.format(nonNC1_total))
print('Sum of MTG ORIG AMT among all = {:.2f}'.format(nc1_total+nonNC1_total))
print('MTG ORIG AMT among NC1 / MTG ORIG AMT of all = {:.2f}%'.format(nc1_total/(nc1_total+nonNC1_total)*100))

Sum of MTG ORIG AMT among NC1 = 30423.10
Sum of MTG ORIG AMT among non-NC1 = 1387799.87
Sum of MTG ORIG AMT among all = 1418222.97
MTG ORIG AMT among NC1 / MTG ORIG AMT of all = 2.15%


In [15]:
print('Desciption of MTG ORIG AMT among NC1:')
nc1['MTG ORIG AMT'].describe()

Desciption of MTG ORIG AMT among NC1:


count    3856.000000
mean        7.889809
std        14.696035
min         0.000100
25%         1.828500
50%         4.258000
75%         9.924000
max       560.470000
Name: MTG ORIG AMT, dtype: float64

## To do
- Look into Bloomberg (Paydown Infomation) <br>
API found, can actually use excel on Bloomberg terminal to get the HIST_UNSUPPORTED_RISK_SHORTFALL data quickly
- Look why payment just suddenly stops instead of gradually decreased and stoped


## Finding duplicatations
The CUSIP in dataset is duplicated. The uniqueness is determined by using both CUSIP and PID. The way of finding duplicated CUSIP is the following:

This line of code shows that CUSIP is not unique in the dataset. Using count is to count the total number of rows, but using nunique is to count unique. I think both method will be handy in different situations. The Falses in result shows that two ways of counting are different.

In [16]:
data.groupby('Label')['CUSIP'].size() == data.groupby('Label')['CUSIP'].nunique()

Label
0                              False
1.4                            False
FE                             False
IOfailing                       True
IOpassMED                      False
IOpassMEY                      False
MED                            False
MEY                            False
NC1(failing)                   False
NC2(z>1 nsf)                   False
NC3(z>1, not paid off, nsf)    False
NMEm                           False
NMEs                            True
Name: CUSIP, dtype: bool

In [17]:
CUSIP_PID_count = data.groupby(by=['CUSIP','PID'])['CUSIP','PID'].nunique().sum()
CUSIP_PID_count

CUSIP    24347
PID      24347
dtype: int64

In [18]:
print('CUSIP and PID agree but does not equal to number of rows. We have {} rows of data'.format(data.shape[0]))
print('The difference is {}'.format(data.shape[0]-CUSIP_PID_count[0]))

CUSIP and PID agree but does not equal to number of rows. We have 24597 rows of data
The difference is 250


Duplicated rows are found below

In [19]:
data.groupby(by=['CUSIP','PID']).size().sort_values(ascending=False).head()

CUSIP      PID    
576434DB7  FC_1440    2
12628LAC4  FC_1840    2
92977TAG7  FC_1890    2
92977TAF9  FC_1890    2
92977TAE2  FC_1890    2
dtype: int64

Theese two rows are he same

In [20]:
data.loc[data['CUSIP'] == '576434DB7']

,Year,PID,Prospectus,Class,norm_class,Name,Current_Balance,Zero-Balance Payment Period Number,Sum Principle Paid,MTG ORIG AMT,...,CUSIP,MTG_TRANCHE_TYP_LONG,Moody Rating,Initial Moody Rating,Bloomberg Composite,HCLB,MTG INT SHRTFLL,HIST INTRST SHRTFLL,Label,NL_fail
3324,2003,FC_1440,MASTR_1225297_0000950136-03-000749.txt,6A2,A,MALT 2003-2 6A2,0.0,1,7.778,7.778,...,576434DB7,"SEQ,AS",WR,Aaa,NR,0.0,NaN,0\0\0,MED,1
3325,2003,FC_1440,MASTR_1225297_0000950136-03-000749.txt,6A2,A,MALT 2003-2 6A2,0.0,1,7.778,7.778,...,576434DB7,"SEQ,AS",WR,Aaa,NR,0.0,NaN,0\0\0,MED,1


data.duplicated() helps us find duplicated rows and True means it is duplicated

In [21]:
data.duplicated().sort_values(ascending=False).head()

4444     True
4493     True
2891     True
4442     True
16595    True
dtype: bool

In [22]:
num_duplicated = data.loc[data.duplicated(keep=False)].shape[0]
print('keep=false shows all duplicated instace so there are {} rows that are duplicated.'.format(num_duplicated))

keep=false shows all duplicated instace so there are 474 rows that are duplicated.


In [23]:
num_to_delete = data.loc[data.duplicated()].shape[0]
print('No keep parameter shows duplicated instace that we should delete so there are {} rows'
      'that are duplications that should be deleted.'.format(num_to_delete))
print('That will give us {}'.format(num_duplicated-num_to_delete))

No keep parameter shows duplicated instace that we should delete so there are 237 rowsthat are duplications that should be deleted.
That will give us 237


Use new variable no_duplicated_data using drop_duplicates()

In [24]:
no_duplicated_data = data.drop_duplicates()

In [25]:
if (((data.shape[0]-CUSIP_PID_count[0])-(num_duplicated-num_to_delete))==0): print('No!! Data rows and CUSIP_PID still not match')
else: print('Total Number Match!!!!')

Total Number Match!!!!


In [26]:
no_duplicated_data.groupby(by=['CUSIP','PID']).size().sort_values(ascending=False).head()

CUSIP      PID    
576434AH7  FC_1509    2
576434AK0  FC_1509    2
86358RX28  FC_2784    2
576434AE4  FC_1509    2
576434AF1  FC_1509    2
dtype: int64

In [27]:
no_duplicated_data.groupby(by=['CUSIP'])['PID'].nunique().sort_values(ascending=False).head()

CUSIP
126671Z90    3
126671Z25    3
126671Z66    3
126671Z58    3
05948KBB4    3
Name: PID, dtype: int64

In [28]:
no_duplicated_data.loc[no_duplicated_data['CUSIP'] == '05948KXX2']

,Year,PID,Prospectus,Class,norm_class,Name,Current_Balance,Zero-Balance Payment Period Number,Sum Principle Paid,MTG ORIG AMT,...,CUSIP,MTG_TRANCHE_TYP_LONG,Moody Rating,Initial Moody Rating,Bloomberg Composite,HCLB,MTG INT SHRTFLL,HIST INTRST SHRTFLL,Label,NL_fail
11167,2005,FC_1378,COUNTRYWIDE_1316390_0000950129-05-000828.txt,2CB1,U,BOAA 2005-2 2CB1,2.8537,paying,22.0169,25.309,...,05948KXX2,"PT,AS",Caa2,Aaa,DDD+,NaN,0.0,-77664.73\26803.1\0,MEY,1
13609,2005,FC_448,BANC_OF_AMERICA_1318762_0001193125-05-034840.htm,2CB1,U,BOAA 2005-2 2CB1,2.8634,paying,22.0085,25.309,...,05948KXX2,"PT,AS",Caa2,Aaa,DDD+,NaN,0.0,-77664.73\26803.1\0,MEY,1


In [29]:
no_duplicated_data.loc[no_duplicated_data['CUSIP'] == '576434AH7']

,Year,PID,Prospectus,Class,norm_class,Name,Current_Balance,Zero-Balance Payment Period Number,Sum Principle Paid,MTG ORIG AMT,...,CUSIP,MTG_TRANCHE_TYP_LONG,Moody Rating,Initial Moody Rating,Bloomberg Composite,HCLB,MTG INT SHRTFLL,HIST INTRST SHRTFLL,Label,NL_fail
482,2002,FC_1509,MASTR_1179092_0000950136-02-002215.txt,AX,A,MALT 2002-1 AX,0.2771,paying,0.0,16.9397,...,576434AH7,"IO,NTL",NaN,NaN,NR,0.0,0.0,0\0\0,IOpassMEY,0
483,2002,FC_1509,MASTR_1179092_0000950136-02-002215.txt,AX,A,MALT 2002-1 AX,0.2802,paying,0.0,16.9397,...,576434AH7,"IO,NTL",NaN,NaN,NR,0.0,0.0,0\0\0,IOpassMEY,0


In [30]:
no_duplicated_data.groupby(by=['CUSIP','PID','Current_Balance']).size().sort_values(ascending=False).head()

CUSIP      PID      Current_Balance
81744Mar3  FC_114   0.0000             2
86358RJ40  FC_386   0.0000             2
86358RN52  FC_2609  0.0000             2
81744MAQ5  FC_114   16.6541            2
576434AE4  FC_1509  0.0000             2
dtype: int64

In [31]:
no_duplicated_data.loc[no_duplicated_data['CUSIP'] == '81744Mar3']

,Year,PID,Prospectus,Class,norm_class,Name,Current_Balance,Zero-Balance Payment Period Number,Sum Principle Paid,MTG ORIG AMT,...,CUSIP,MTG_TRANCHE_TYP_LONG,Moody Rating,Initial Moody Rating,Bloomberg Composite,HCLB,MTG INT SHRTFLL,HIST INTRST SHRTFLL,Label,NL_fail
15532,2006,FC_114,SEQUOIA_1405457_0001144204-07-038661.htm,2BA2,B,SEMT 2007-3 2BA2,0.0,25,1.1852,3.378,...,81744Mar3,"CSTR,SSUP,AS",NR,NR,NR,2.1928,0.0,0\0\0,"NC3(z>1, not paid off, nsf)",0
15536,2006,FC_114,SEQUOIA_1405457_0001144204-07-038661.htm,2BA2,U,SEMT 2007-3 2BA2,0.0,25,1.1852,3.378,...,81744Mar3,"CSTR,SSUP,AS",NR,NR,NR,2.1928,0.0,0\0\0,"NC3(z>1, not paid off, nsf)",0


In [32]:
no_duplicated_data.groupby(by=['CUSIP','PID','Current_Balance','norm_class']).size().sort_values(ascending=False).head()

CUSIP      PID      Current_Balance  norm_class
576434AE4  FC_1509  0.0000           A             2
BCC2BQMX0  FC_704   14.2665          A             1
12669D6N3  FC_1219  0.0000           A             1
12669D6Q6  FC_1219  0.0503           B             1
12669D6R4  FC_1219  0.0176           B             1
dtype: int64

In [33]:
no_duplicated_data.loc[no_duplicated_data['CUSIP'] == '576434AE4']

,Year,PID,Prospectus,Class,norm_class,Name,Current_Balance,Zero-Balance Payment Period Number,Sum Principle Paid,MTG ORIG AMT,...,CUSIP,MTG_TRANCHE_TYP_LONG,Moody Rating,Initial Moody Rating,Bloomberg Composite,HCLB,MTG INT SHRTFLL,HIST INTRST SHRTFLL,Label,NL_fail
491,2002,FC_1509,MASTR_1179092_0000950136-02-002215.txt,A5,A,MALT 2002-1 A5,0.0,1,25.65,25.65,...,576434AE4,NAS,NaN,NaN,NR,0.0,NaN,0\0\0,MED,0
492,2002,FC_1509,MASTR_1179092_0000950136-02-002215.txt,A5,A,MALT 2002-1 A5,0.0,1,25.65,25.65,...,576434AE4,NON-ACCELERATED SECURITY,NaN,NaN,NR,0.0,NaN,0\0\0,MED,0


In [34]:
no_duplicated_data.groupby(by=['CUSIP','PID','Current_Balance','norm_class','MTG_TRANCHE_TYP_LONG']).size().sort_values(ascending=False).head()

CUSIP      PID      Current_Balance  norm_class  MTG_TRANCHE_TYP_LONG
BCC2BQMX0  FC_704   14.2665          A           STEP,AFC,AS             1
12669DA65  FC_3168  0.0000           A           SEQ,AS                  1
12669D6R4  FC_1219  0.0176           B           SUB,CSTR,NAS            1
12669D7J1  FC_1845  0.0000           B           SUB,NAS                 1
12669D7K8  FC_1845  0.0000           B           SUB,NAS                 1
dtype: int64

Here is the way to find what CUSIPs are duplicated. (Here only shows the top five and bottom five results which are all >1). The series can be exported using pandas function

In [35]:
#Get series of CUSIP and get count > 1
series = data.groupby(by=['CUSIP']).size().sort_values(ascending=False)
series = series.loc[series > 1]

In [36]:
series.head()

CUSIP
94974SAB9    3
126671Y91    3
05948KAS8    3
1266712B1    3
1266712F2    3
dtype: int64

In [37]:
series.tail()

CUSIP
576434HV9    2
576434HU1    2
576434HT4    2
576433QQ2    2
576433QR0    2
dtype: int64